In [ ]:
import psycopg2
import pandas as pd
import 
import matplotlib
import sqlalchemy
from sqlalchemy import create_engine



In [ ]:
user = 'postgres'
password = 'password'
host = 'localhost'
port = 5432
dbname = 'employees'
engine = create_engine(f'postgres://{user}:{password}@{host}:{port}/{dbname}')
engine = engine.connect()
conn = engine.connect()

In [ ]:
#Read all data form sql files
departments_df = pd.read_sql('SELECT * FROM departments', conn)
dept_emp_df = pd.read_sql('SELECT * FROM dept_emp', conn)
dept_manager_df = pd.read_sql('SELECT * FROM dept_manager', conn)
employees_df = pd.read_sql('SELECT * FROM employees', conn)
salaries_df = pd.read_sql('SELECT * FROM salaries', conn)
titles_df = pd.read_sql('SELECT * FROM titles', conn)

In [ ]:
#Rename column
employees_df.rename(columns = {'emp_title_id':'title_id'}, inplace = True) 
employees_df.head()

In [ ]:
 #Plot histogram salary range for employee
ax = salaries_df.hist(column='salary', bins=25, grid=False, figsize=(12,8), color='red', zorder=2, rwidth=0.9)

ax = ax[0]
for x in ax:

    # Set the limits on the y-axis
    x.set_ylim(0, 120000)
    
    # Draw horizontal axis lines
    vals = x.get_yticks()
    for tick in vals:
        x.axhline(y=tick, linestyle='solid', alpha=0.4, color='#777777', zorder=1)

    # Set title
    x.set_title(f"\n Salary Range Histogram - Pewlett-Hackard 1985-2000 \n", weight='bold', size=16)

    # Set x- and y-axis labels
    x.set_xlabel("Salary", labelpad=20, weight='bold', size=12)
    x.set_ylabel("Frequency", labelpad=20, weight='bold', size=12)

    # Format x- and y-axis labels
    x.xaxis.set_major_formatter(StrMethodFormatter('${x:,g}'))
    x.yaxis.set_major_formatter(StrMethodFormatter('{x:,g}'))

In [ ]:
# Merge 3 dataframes/SQLtables to be able to plot Average Salary by Employee Title
df1 = pd.merge(employees_df, titles_df, on="title_id")
df2 = pd.merge(df1, salaries_df, on="emp_no")
df2.head()

In [ ]:
# Perform Groupby and calculate average salary by employee title
avg_salary_by_title = df2.groupby("title")["salary"].mean()
avg_salary_by_title_df = pd.DataFrame(avg_salary_by_title).reset_index()
avg_salary_by_title_df.rename(columns = {'title':'Employee Title', 'salary':'Average Salary'}, inplace = True) 
avg_salary_by_title_df

In [ ]:
# Plot bar graph of Average Salary by employees
ax = avg_salary_by_title_df.plot.bar(x='Employee Title',
                                     y='Average Salary',
                                     legend=None,
                                     rot=0,
                                     grid=False,
                                     color=['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'],
                                     alpha=1.0,
                                     zorder=2,
                                     figsize=(12,8))

# Set the limits on the y-axis
ax.set_ylim(0, 60000)

# Draw horizontal axis lines
vals = np.arange(0, 60000, 10000)
for tick in vals:
    ax.axhline(y=tick, linestyle='solid', alpha=0.4, color='#777777') #, zorder=1)

# Set title
ax.set_title(f"\n Average Salary by Employee Title - Pewlett-Hackard 1985-2000 \n", weight='bold', size=16)

# Set x- and y-axis labels
ax.set_xlabel("Employee Title", labelpad=20, weight='bold', size=12)
ax.set_ylabel("Average Salary", labelpad=20, weight='bold', size=12)

# Format y-axis labels
ax.yaxis.set_major_formatter(StrMethodFormatter('${x:,g}'))